In [ ]:
from os import path
import pickle

import astropy.coordinates as coord
from astropy.table import Table
import astropy.units as u
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
#plt.style.use('apw-notebook')
%matplotlib inline
import sqlutil
from scipy.stats import binned_statistic_dd
import testXD
from dustmaps.sfd import SFDQuery
from astropy.coordinates import SkyCoord
import astropy.units as units

In [ ]:
def queryDatabase(query, columns, post_query, cache_file='db.pickle'):

    #cache_file = "reducedPM2.pickle"

    if not path.exists(cache_file):
        res = sqlutil.get(query.format(','.join(columns), post_query),
                          db='wsdb', host='cappc127.ast.cam.ac.uk', 
                          user=user, password=password, 
                          asDict=True)

        with open(cache_file, "wb") as f:
            pickle.dump(res, f)
        
    else:
        with open(cache_file, "rb") as f:
            res = pickle.load(f)
    return res

In [ ]:
def dust(l, b):
    c = SkyCoord(l, b, frame='galactic')
    sfd = SFDQuery()
    dust = sfd(c)
    return dust

In [ ]:
def reducedPM(pmra, pmdec, mag):
    pm = np.sqrt(pmra**2. + pmdec**2.)
    return mag + 5.*np.log10(pm) + 5.

In [ ]:
dustCoeff = {'B': 3.626,
             'V': 2.742,
             'g': 3.303,
             'r': 2.285,
             'i': 1.698,
             'J': 0.709,
             'H': 0.449,
             'K': 0.302,
             'G': 2.55}

In [ ]:
columns = '*'
query = "Select {0} FROM gaia_dr1.gaia_source {1}"
cuts = 'LIMIT 10'
res = queryDatabase(query, columns, cuts, cache_file='gaiaListColumns')
tbl = Table(res)
tbl.keys()

In [ ]:
columns = '*'
query = "Select {0} FROM gaia_dr1_aux.gaia_source_sdssdr9_xm_new {1}"
cuts = "LIMIT 10"
res = queryDatabase(query, columns, cuts, cache_file='sdssListColumns')
tbl = Table(res)
tbl.keys()

In [ ]:
columns = '*'
query = "Select {0} FROM gaia_dr1_aux.gaia_source_2mass_xm {1}"
cuts = "LIMIT 10"
res = queryDatabase(query, columns, cuts, cache_file='tmassListColumns')
tbl = Table(res)
tbl.keys()

In [ ]:
sdsscolumns = [
    'random_index', 'source_id', 'ra', 'dec', 'l', 'b', 
    'pmra_new', 'pmdec_new', 'gs_pmra', 'gs_pmdec', 'extinction_u', 
    'extinction_g', 'extinction_r', 'extinction_i', 'extinction_z',
    'psfmag_u', 'psfmag_g', 'psfmag_r', 'psfmag_i', 'psfmag_z', 'phot_g_mean_mag']
query = "Select {0} FROM gaia_dr1_aux.gaia_source_sdssdr9_xm_new {1}"
cuts = """WHERE MOD(random_index, 10) = 0 AND 
sqrt(power(2.5/log(10)*phot_g_mean_flux_error/phot_g_mean_flux,2)) <= 0.05 AND
abs(b) > 30
"""
res = queryDatabase(query, sdsscolumns, cuts, cache_file='gaiasdssHalo.pkl')
sdsstbl = Table(res)


In [ ]:
tmasscolumns = [
        'random_index', 'source_id', 'ra', 'dec', 'l', 'b', 
    'gt_pmra', 'gt_pmdec', 
    'j_m', 'h_m', 'k_m', 'pmra', 'pmdec',
    'phot_g_mean_mag']
query = "Select {0} FROM gaia_dr1_aux.gaia_source_2mass_xm {1}"
cuts = """WHERE MOD(random_index, 10) = 0 AND 
sqrt(power(2.5/log(10)*phot_g_mean_flux_error/phot_g_mean_flux,2)) <= 0.05
AND abs(b) > 30
"""
res = queryDatabase(query, tmasscolumns, cuts, cache_file='gaia2massHalo.pkl')
tmasstbl = Table(res)

In [ ]:
sdssdust = dust(sdsstbl['l']*units.deg, sdsstbl['b']*units.deg)

In [ ]:
tmassdust = dust(tmasstbl['l']*units.deg, tmasstbl['b']*units.deg)

In [ ]:
sdsstbl['reduced_pm'] = reducedPM(sdsstbl['pmra_new'], sdsstbl['pmdec_new'], sdsstbl['phot_g_mean_mag'] - sdssdust*dustCoeff['G'])
sdsscolor = sdsstbl['psfmag_g'] - sdssdust*dustCoeff['g'] - (sdsstbl['psfmag_i'] - sdssdust*dustCoeff['i'])

In [ ]:
tmasstbl['reduced_pm'] = reducedPM(tmasstbl['phot_g_mean_mag'] - tmassdust*dustCoeff['G']+ 5.*np.log10(tmasspm) + 5.
tmasscolor = tmasstbl['j_m'] - tmassdust*dustCoeff['J']- (tmasstbl['k_m'] - tmassdust*dustCoeff['K'])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,7.5))
sdssgood = (np.abs(sdsscolor) < 10) & (sdsstbl['reduced_pm'] < 40) & (sdsstbl['reduced_pm'] > 0)
tmassgood = (np.abs(tmasscolor) < 10) & (tmasstbl['reduced_pm'] < 40) & (tmasstbl['reduced_pm'] > 0)
ax[0].hist2d(sdsscolor[sdssgood], sdsstbl['reduced_pm'][sdssgood], bins=500, cmap='Greys', norm=mpl.colors.LogNorm()) #, s=1, lw=0, alpha=0.5)
ax[0].set_ylim(40, 15)
ax[0].set_xlim(-1, 4)
ax[0].set_xlabel('g-i')
ax[0].set_ylabel('Reduced PM G')
#ax[0].colorbar()
ax[1].hist2d(tmasscolor[tmassgood], tmasstbl['reduced_pm'][tmassgood], bins=500, cmap='Greys', norm=mpl.colors.LogNorm())
#ax[1].colorbar()
ax[1].set_ylim(40, 15)
ax[1].set_xlim(-1, 4)
ax[1].set_xlabel('j-k')
ax[1].set_ylabel('Reduced PM G')
plt.savefig('reducedPM.pdf')

In [ ]:
dustCoeff = {'B': 0.0,
             'V': 0.0,
             'g': 0.0,
             'r': 0.0,
             'i': 0.0,
             'J': 0.0,
             'H': 0.0,
             'K': 0.0,
             'G': 0.0}

In [ ]:
sdsstbl['reduced_pm'] = sdsstbl['phot_g_mean_mag'] - sdssdust*dustCoeff['G'] + 5.*np.log10(sdsspm) + 5.
sdsscolor = sdsstbl['psfmag_g'] - sdssdust*dustCoeff['g'] - (sdsstbl['psfmag_i'] - sdssdust*dustCoeff['i'])

In [ ]:
tmasstbl['reduced_pm'] = tmasstbl['phot_g_mean_mag'] - tmassdust*dustCoeff['G']+ 5.*np.log10(tmasspm) + 5.
tmasscolor = tmasstbl['j_m'] - tmassdust*dustCoeff['J']- (tmasstbl['k_m'] - tmassdust*dustCoeff['K'])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,7.5))
sdssgood = (np.abs(sdsscolor) < 10) & (sdsstbl['reduced_pm'] < 40) & (sdsstbl['reduced_pm'] > 0)
tmassgood = (np.abs(tmasscolor) < 10) & (tmasstbl['reduced_pm'] < 40) & (tmasstbl['reduced_pm'] > 0)
ax[0].hist2d(sdsscolor[sdssgood], sdsstbl['reduced_pm'][sdssgood], bins=500, cmap='Greys', norm=mpl.colors.LogNorm()) #, s=1, lw=0, alpha=0.5)
ax[0].set_ylim(40, 15)
ax[0].set_xlim(-1, 4)
ax[0].set_xlabel('g-i')
ax[0].set_ylabel('Reduced PM G')
#ax[0].colorbar()
ax[1].hist2d(tmasscolor[tmassgood], tmasstbl['reduced_pm'][tmassgood], bins=500, cmap='Greys', norm=mpl.colors.LogNorm())
#ax[1].colorbar()
ax[1].set_ylim(40, 15)
ax[1].set_xlim(-1, 2)
ax[1].set_xlabel('j-k')
ax[1].set_ylabel('Reduced PM G')
plt.savefig('reducedPM_nodustHalo.pdf')